In [27]:
import pandas as pd
import numpy as np

In [28]:
# X data>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
x_train = pd.read_parquet("x_train_knn_simple.parquet")
x_valid = pd.read_parquet("x_valid_knn_simple.parquet")
x_test = pd.read_parquet("x_test_knn_simple.parquet")

# Y data >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
y_train = pd.read_parquet("scaled_y_train_knn.parquet")
y_valid = pd.read_parquet("scaled_y_valid_knn.parquet")
y_test = pd.read_parquet("scaled_y_test_knn.parquet")

In [29]:
from time import time
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
from sklearn.cross_decomposition import PLSRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import GradientBoostingRegressor, ExtraTreesRegressor, RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error, explained_variance_score, mean_absolute_percentage_error
from sklearn.svm import SVR
import warnings
warnings.filterwarnings("ignore")
from sklearn.feature_selection import RFECV

In [30]:
model = [XGBRegressor(),LGBMRegressor(),DecisionTreeRegressor()]

In [31]:
rfecv = RFECV(estimator=XGBRegressor(),min_features_to_select=10,scoring="neg_mean_absolute_error",n_jobs=-1)

In [32]:
_ = rfecv.fit(x_train,y_train)

In [33]:
xgbmask = rfecv.support_

In [34]:
lgrfecv = RFECV(estimator=LGBMRegressor(),min_features_to_select=10,scoring="neg_mean_absolute_error",n_jobs=-1)

In [35]:
_ = lgrfecv.fit(x_train,y_train)

In [36]:
lgbmask = lgrfecv.support_

In [37]:
randrfecv = RFECV(estimator=DecisionTreeRegressor(),min_features_to_select=10,scoring="neg_mean_absolute_error",n_jobs=-1)

In [38]:
_ = randrfecv.fit(x_train,y_train)

In [39]:
randmask = randrfecv.support_

In [40]:
votes = np.sum([xgbmask,lgbmask,randmask],axis=0)

In [41]:
votes

array([2, 2, 2, 3, 3, 2, 2, 3, 2, 3, 3, 2, 3, 3, 2, 2, 2, 2, 1, 0, 2, 3,
       3, 3, 3, 2, 1, 1, 3, 1, 1, 1, 0, 0, 3, 3, 3, 3, 3, 3, 1, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3])

In [42]:
finalMask = votes==3

In [43]:
finalMask

array([False, False, False,  True,  True, False, False,  True, False,
        True,  True, False,  True,  True, False, False, False, False,
       False, False, False,  True,  True,  True,  True, False, False,
       False,  True, False, False, False, False, False,  True,  True,
        True,  True,  True,  True, False,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True])

In [44]:
x_train.loc[:,finalMask].shape

(53402, 31)

In [45]:
x_train = x_train.loc[:,finalMask]

In [46]:
x_train.shape

(53402, 31)

In [47]:
x_valid = x_valid.loc[:,finalMask]
x_test = x_test.loc[:,finalMask]

In [48]:
x_test.shape

(11088, 31)

In [51]:
x_valid.head()

,floor_area,energy_star_rating,heating_degree_days,snowfall_inches,snowdepth_inches,days_below_30F,days_below_20F,age,Q1,Q2,...,cat__facility_type_Health,cat__facility_type_Industry,cat__facility_type_Lodging,cat__facility_type_Multifamily,cat__facility_type_Office,cat__facility_type_Public_area,cat__facility_type_Public_safety,cat__facility_type_Retail,cat__facility_type_Service,cat__facility_type_Warehouse
0,0.516336,0.570565,-0.220167,0.411490,0.131023,-0.252631,-0.225766,0.725684,0.433672,-0.130958,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,-0.217012,0.791787,0.664602,1.106299,1.116781,0.637321,0.611933,-0.016611,-0.725460,-0.049377,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.856405,-0.838008,0.361692,0.078679,-0.142157,0.101718,-0.225766,-0.383934,-0.129259,-0.121076,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,-0.531507,1.703350,-2.089166,-1.775379,-1.651850,-1.032144,-1.716685,0.194060,1.585022,1.893293,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0.416354,-0.324436,0.361692,0.078679,-0.142157,0.101718,-0.225766,0.900075,-0.129259,-0.121076,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [49]:
y_test.shape

(11088, 1)

In [50]:
x_train.to_parquet('x_train_rfe.parquet')
x_valid.to_parquet('x_valid_rfe.parquet')
x_test.to_parquet('x_test_rfe.parquet')